In [1]:
import tensorflow as tf
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, accuracy_score

Create model

In [2]:
# Define the model

def create_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    
    # First layer
    conv1 = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1),kernel_regularizer=tf.keras.regularizers.l2(0.001) ,padding='valid')(inputs)
    batchnorm1 = tf.keras.layers.BatchNormalization()(conv1)
    relu1 = tf.keras.layers.ReLU()(batchnorm1)
    maxpool1 = tf.keras.layers.MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same')(relu1)
    
    # Second layer
    conv2 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1),kernel_regularizer=tf.keras.regularizers.l2(0.001) ,padding='valid')(maxpool1)
    batchnorm2 = tf.keras.layers.BatchNormalization()(conv2)
    relu2 = tf.keras.layers.ReLU()(batchnorm2)
    maxpool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(relu2)
    
    # Third layer
    conv3 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1),kernel_regularizer=tf.keras.regularizers.l2(0.001), padding='valid')(maxpool2)
    batchnorm3 = tf.keras.layers.BatchNormalization()(conv3)
    relu3 = tf.keras.layers.ReLU()(batchnorm3)
    avgpool = tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(relu3)
    
    # Flatten the output of the last convolutional layer
    flatten = tf.keras.layers.Flatten()(avgpool)
    
    # Fully connected layers
    fc1 = tf.keras.layers.Dense(256, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001))(flatten)
    fc2 = tf.keras.layers.Dense(256, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001))(fc1)

    # Output layer (binary classification)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(fc2)
    
    # Create the model
    model = tf.keras.Model(inputs=inputs, outputs=output)
    return model

# Example usage
input_shape = (160, 160, 1)  

model = create_model(input_shape)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 160, 160, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 158, 158, 16)      160       
                                                                 
 batch_normalization (Batch  (None, 158, 158, 16)      64        
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 158, 158, 16)      0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 40, 40, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 38, 38, 32)        4640  

Loading images

In [4]:
folder1_path = '.\persons\person1'
folder2_path = '.\persons\person2'

def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, filename),cv2.COLOR_BGR2GRAY)
        if img is not None:
            images.append(img)
    return images

images_for_person1 = load_images_from_folder(folder1_path)
images_for_person2 = load_images_from_folder(folder2_path)


indices_to_separate = [2,5,10,11,12,13,15,39,40,41,42,44]

#Separating indices for person 1
testing_indices_person1 = indices_to_separate.copy()
testing_images_person1 = [images_for_person1[i] for i in testing_indices_person1]
remaining_indices_person1 = np.setdiff1d(np.arange(len(images_for_person1)), testing_indices_person1)
training_images_person1 = [images_for_person1[i] for i in remaining_indices_person1]

# Separating indices for person 2
testing_indices_person2 = indices_to_separate.copy()
testing_images_person2 = [images_for_person2[i] for i in testing_indices_person2]
remaining_indices_person2 = np.setdiff1d(np.arange(len(images_for_person2)), testing_indices_person2)
training_images_person2 = [images_for_person2[i] for i in remaining_indices_person2]

Data augmentation

In [ ]:
# Function to perform image sharpening
def sharpen_image(image):
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    sharpened = cv2.filter2D(image, -1, kernel)
    return sharpened

# Function to perform image flipping
def flip_image(image):
    flipped = cv2.flip(image, 1)  # 1 for vertical flipping
    return flipped

# Perform data augmentation for person 1
augmented_training_images_person1 = []
for image in training_images_person1:
    sharpened_image = sharpen_image(image)
    flipped_image = flip_image(image)
    augmented_training_images_person1.append(sharpened_image)
    augmented_training_images_person1.append(flipped_image)

# Perform data augmentation for person 2
augmented_training_images_person2 = []
for image in training_images_person2:
    sharpened_image = sharpen_image(image)
    flipped_image = flip_image(image)
    augmented_training_images_person2.append(sharpened_image)
    augmented_training_images_person2.append(flipped_image)
             

# Convert lists to numpy arrays
augmented_training_images_person1 = np.array(augmented_training_images_person1)
augmented_training_images_person2 = np.array(augmented_training_images_person2)

Prepare data to feeding

In [ ]:
#concatenate the 2 training array and labeling crossponding index in y_train
X_train = np.concatenate((augmented_training_images_person1, augmented_training_images_person2))
y_train = np.concatenate((
    np.zeros(len(augmented_training_images_person1)),
    np.ones(len(augmented_training_images_person2))
))

# Shuffle the data
shuffle_indices = np.random.permutation(len(X_train))
X_train_shuffled = X_train[shuffle_indices]
y_train_shuffled = y_train[shuffle_indices]